## Compiling functions with Cython

In [1]:
import pyximport

In [2]:
%load_ext Cython

Function: $y = (x^2 + 3) / 4.5$

In [3]:
def my_op(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [4]:
%%cython

def my_cop(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [5]:
my_op

<function __main__.my_op(data)>

In [6]:
my_cop

<function _cython_magic_5f50c224c320b4c4af3f7ea75a5b42ca.my_cop>

In [7]:
%timeit my_op([1,2,3,4,5])

2.23 µs ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
%timeit my_cop([1,2,3,4,5])

1.33 µs ± 2.93 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
import numpy

In [10]:
numpy.array([1,2,3,4,5])

array([1, 2, 3, 4, 5])

In [11]:
d = numpy.array([1,2,3,4,5])
( d ** 2 + 3 ) / 4.5

array([0.88888889, 1.55555556, 2.66666667, 4.22222222, 6.22222222])

In [12]:
%timeit ( d ** 2 + 3 ) / 4.5

3.8 µs ± 52.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
bigdata = numpy.random.randn(1000000)
bigdata.dtype

dtype('float64')

In [14]:
%timeit my_op(bigdata.copy())

1.25 s ± 22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit ( bigdata.copy() ** 2 + 3 ) / 4.5

5.4 ms ± 60.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%timeit my_cop(bigdata.copy())

1.07 s ± 30.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%cython
cimport numpy as np

def my_cop(np.ndarray[np.float_t, ndim=1] data):
    cdef int i

    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [18]:
my_cop

<function _cython_magic_f177c48b24ccc246b5140bf16197bc32.my_cop>

In [19]:
%timeit my_cop(bigdata.copy())

6.54 ms ± 31.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Just-in-time optimization with numba

In [20]:
from numba import jit

In [21]:
@jit
def my_nbop(data):
    for i in range(len(data)):
        data[i] = ( data[i] ** 2 + 3 ) / 4.5
    return data

In [22]:
my_nbop

CPUDispatcher(<function my_nbop at 0x2afd6d766a60>)

In [23]:
%timeit my_nbop([1,2,3,4,5])

/panfs/roc/msisoft/anaconda/python3-2020.07-mamba/lib/python3.8/site-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'data' of function 'my_nbop'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-21-de065de12a6b>", line 2:
@jit
def my_nbop(data):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


19.1 µs ± 8.17 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit my_nbop(bigdata.copy())

3.81 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
data2 = numpy.arange(1000000)

In [30]:
data2[0:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [31]:
%timeit my_nbop(data2)

3.18 ms ± 17.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Expression evaluators with numexpr

In [32]:
from numexpr import evaluate

In [33]:
smalldata = numpy.array([1,2,3,4,5])

In [34]:
evaluate("(smalldata**2 + 3) / 4.5")

array([0.88888889, 1.55555556, 2.66666667, 4.22222222, 6.22222222])

In [35]:
%timeit evaluate("(bigdata**2 + 3) / 4.5")

901 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
